In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = requests.utils.default_headers()
headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [ ]:
dates = pd.date_range(start='01/01/2022', end='30/04/2022')
dates = dates.strftime('%d.%m.%Y').to_list()
hours = np.arange(24)

In [ ]:
def df_from_url(date, hour):
    main_url = 'https://www.ilmateenistus.ee/ilm/ilmavaatlused/vaatlusandmed/tunniandmed/'
    sub_url = f'?filter[date]={date}&filter[hour]={hour}'
    req = requests.get(main_url + sub_url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    table = soup.find_all('table')
    df = pd.read_html(str(table), decimal=',', thousands='.')[0]
    if sub_url != 'maxmin-ohutemp':
        df.columns = [x+' '+y if x!=y else x for x, y in df.columns]
    df = df.loc[df['Jaam']=='Tallinn-Harku']
    df = df.drop(columns=['Jaam'])
    df['date'] = date
    df['hour'] = hour
    return df

In [ ]:
df_list = []

for date in dates:
    for hour in hours:
        print(f'{date} {hour}')
        df_list.append(df_from_url(date, hour))

In [ ]:
df_2022 = pd.concat(df_list, ignore_index=False)

In [ ]:
df_2022.to_csv('2022.csv', index=False)

In [ ]:
df_2022